# Cisco DevNet - RestAPI 完整实验🏀

## LAB 1. basic-labs 🏁

In [27]:
import json
import sys
import threading
import time
from helper import apicem
from helper.tabulate import tabulate
from helper.apicem_config import APICEM_IP, VERSION, USERNAME, PASSWORD

### 1.1 Get user

In [ ]:
# Controller ip, username and password are defined in apicem_config.py
# The get() function is defined in apicem.py
# Get token function is called in get() function
try:
    resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="user")
    response_json = resp.json() # Get the json-encoded content from response
    print (json.dumps(response_json,indent=4),'\n') # Convert "response_json" object to a JSON formatted string and print it out
except:
    print ("Something wrong with GET /user request")
    sys.exit()

# Parsing raw response to list out all users and their role
for item in response_json["response"]:
    for item1 in item["authorization"]:
        print ("User \'%s\', role is the %s."%(item["username"],(item1["role"])[5:]))

# [5:] = skip first 5 characters of string item1["role"]

### 2.2 Get network device list

In [ ]:
device = []
try:
    # The request and response of "GET /network-device" API
    resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device")
    status = resp.status_code
    # Get the json-encoded content from response
    response_json = resp.json()
    # All network-device detail is in "response"
    device = response_json["response"]

    # Try un-comment the following line to see what we get

    # print(json.dumps(device,indent=4))
except:
    print ("Something wrong, cannot get network device information")
    sys.exit()

if status != 200:
    print (resp.text)
    sys.exit()

if device == [] :   # Response is empty, no network-device is discovered.
    print ("No network device found !")
    sys.exit()

device_list = []
# Now extract host name, ip and type to a list. Also add a sequential number in front
i=0
for item in device:
    i+=1
    device_list.append([i,item["hostname"],item["managementIpAddress"],item["type"],item["instanceUuid"]])

# We use tabulate module here to print a nice table format. You should use "pip" tool to install in your local machine
# For the simplicity we just copy the source code in working directory without  installing it.
# Not showing id to user, it's just a hex string
print (tabulate(device_list, headers=['number','hostname','ip','type'],tablefmt="rst"))

### 1.3 Get network device id config

In [ ]:
# Print out device list for user to select
device = []
try:
    resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device") # The response (result) from "GET /network-device" request
    status = resp.status_code
    response_json = resp.json() # Get the json-encoded content from response
    device = response_json["response"] # Network-device
except:
    print ("Something wrong, cannot get network device information")
    sys.exit()

if status != 200:
    print (resp.text)
    sys.exit()

# Make sure there is at least one network device
if device == [] :   # if response is not empty
    print ("No network device was found !")
    sys.exit()

# Device found
device_list = []
# Extracting attributes
# Add a counter to an iterable
i=0
for item in device:
    i+=1
    device_list.append([i,item["hostname"],item["managementIpAddress"],item["type"],item["instanceUuid"]])
# Show all network devices under this APIC-EM's management
# Pretty print tabular data, needs 'tabulate' module
print (tabulate(device_list, headers=['number','hostname','ip','type'],tablefmt="rst"),'\n')

print ("*** Please note that some devices may not be able to show configuration for various reasons. ***\n")

# Ask user input
# Find out network device id for network device with ip or hostname, index 4 is the device id
# In the loop until 'id' is assigned or user select 'exit'

id = ""
device_id_idx = 4
while True:
    user_input = input('=> Select a number for the device from above to show IOS config: ')
    user_input= user_input.lstrip() # Ignore leading space
    if user_input.lower() == 'exit':
        sys.exit()
    if user_input.isdigit():
        if int(user_input) in range(1,len(device_list)+1):
            id = device_list[int(user_input)-1][device_id_idx]
            break
        else:
            print ("Oops! number is out of range, please try again or enter 'exit'")
    else:
        print ("Oops! input is not a digit, please try again or enter 'exit'")
# End of while loop

# Get IOS configuration API
try:
    resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device/"+id+"/config")
    status = resp.status_code
except:
    print ("Something wrong with GET network-device/"+id+"/config !\n")
    sys.exit()
try:
    response_json = resp.json()
    # Replace "\r\n" to "\n" to remove extra space line (Carriage Return)
    print (response_json["response"].replace("\r\n","\n"))
except:
    # For some reason IOS configuration is not returned
    if status == 204:
        print ("No Content in response of GET /network-device/id/config !")
    else:
        print ("Something wrong in response of GET /network-device/id/config!\n")
        print ("Response:\n",json.dumps(response_json,indent = 4))

### 1.4 Get network device id config interface

In [ ]:
# Print out device list for user to select

device = []
try:
    resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device") # The response (result) from "GET /network-device" request
    status = resp.status_code
    response_json = resp.json() # Get the json-encoded content from response
    device = response_json["response"] # Network-device
except:
    print ("Something wrong, cannot get network device information")
    sys.exit()
if status != 200:
    print (resp.text)
    sys.exit()

# Make sure there is at least one network device
if device == []:
    print ("No network device found !")
    print (resp.text)

# Ask user's input - What to display ? Interfaces list(1) or IOS config(2) ?
while True:
    user_input = input('=> Please enter \n1: To get list of interfaces for the given device ID\n2: To get IOS configuration for the given device ID\nEnter your selection: ' )
    user_input= user_input.lstrip() # ignore leading space
    if user_input.lower() == 'exit':
        sys.exit()
    if user_input.isdigit():
        if user_input in {'1','2'}:
            break
        else:
            print ("Sorry, wrong selection, please try again to select 1 or 2 or enter 'exit'!")
    else:
       print ("Oops! input is not a digit, please try again or enter 'exit'")
# End of while loop

# Device found
device_list = []
# Extracting attributes
# Add a counter to an iterable
i=0
for item in device:
    i+=1
    device_list.append([i,item["hostname"],item["managementIpAddress"],item["type"],item["instanceUuid"]])
    #Not showing id to user, it's just a hex string

# Show all network devices under this APIC-EM's management
# Pretty print tabular data, needs 'tabulate' module
# Not showing id to user, it's just a hex string
print (tabulate(device_list, headers=['number','hostname','ip','type'],tablefmt="rst"),'\n')

# Ask user's input
# Find out network device id for network device with ip or hostname, index 4 is device id
# In the loop until 'id' is assigned or user select 'exit'
id = ""
device_id_idx = 4
while True:
    if user_input == '1':
        print ("*** Please note that some devices may not be able to show interface info for various reasons. ***\n")
        user_input2 = input('=> Select a number for the device from above to show Interface: ')
    else:
        print ("*** Please note that some devices may not be able to show configuration for various reasons. ***\n")
        user_input2 = input('=> Select a number for the device from above to show IOS config: ')
    user_input2= user_input2.lstrip() # Ignore leading space
    if user_input2.lower() == 'exit':
        sys.exit()
    if user_input2.isdigit(): # Check if the user's input is a digit
        if int(user_input2) in range(1,len(device_list)+1): # Check if input is within range
            id = device_list[int(user_input2)-1][device_id_idx]
            break
        else:
            print ("Oops! number is out of range, please try again or enter 'exit'")
    else:
        print ("Oops! input is not a digit, please try again or enter 'exit'")
# End of while loop

# Show interface or IOS config
if user_input == '1':
    # Get interface list
    selected_api  =  "interface/network-device/"+id
else:
    # Get IOS configuration
    selected_api =  "network-device/"+id+"/config"
# GET api request
try:
    resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api=selected_api)
    status = resp.status_code
except:
    print ("Something wrong with GET %s\n"%s)
    sys.exit()

try:
    response_json = resp.json()
    if user_input == '1': # Interface list
        print ("Response:\n",json.dumps(response_json,indent = 4))
    if user_input == '2': # IOS configuration
        # Replace "\r\n" to "\n" to remove extra space line (Carriage Return)
        print (response_json["response"].replace("\r\n","\n"))
except:
    if status == 204:
        print ("No Content in response of GET %s"%selected_api)
    else:
        print ("Something wrong in response of GET %s!\n"%selected_api)
        print ("Response:\n",json.dumps(response_json,indent = 4))

## LAB 2. Path trace labs 👩‍🏫

### 2.1 Get host

In [21]:
def get_host():
    """
    This function returns a tabular list of all hosts that are connected to APIC-EM network devices.
    Return:
    ------
    list: a list of all hosts and network devices with a number tag
    """
    host_list=[]
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="host") # The get() function is the simplified version for "get" function in requests module, defined in apicem.py
        response_json = resp.json() # Get the json-encoded content from response
        print ("Status: ",resp.status_code)  # This is the http request status
    except:
        print ("Something wrong with GET /host request!")
        return host_list
    # Now create a list of host summary
    i=0
    for item in response_json["response"]:
        i+=1
        host_list.append([i,item["hostIp"],item["hostType"],item["connectedNetworkDeviceIpAddress"]])
    return host_list

host=get_host()
print (tabulate(host,headers=['number','host IP','type','connected to network device'],tablefmt="rst"))


Executing GET 'https://sandboxapicem.cisco.com/api/v1/host'

GET 'host' Status:  200 

Status:  200
========  ===========  ========  =============================
  number  host IP      type      connected to network device
========  ===========  ========  =============================
       1  10.1.15.117  wireless  10.1.14.3
       2  10.2.1.22    wired     10.2.1.17
       3  10.1.12.20   wired     10.1.12.1
========  ===========  ========  =============================


### 2.2 Get host network device iplist

In [28]:
def get_host_and_device():
    """
    This function returns a list of all hosts and network devices with a number tag.

    Return:
    -------
    list: a list of all hosts and network devices with a number tag
    """
    ip_list=[]
    idx=0
    # Get host
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="host")
        print ("Status of GET /host: ",resp.status_code)  # This is the http request status
        response_json = resp.json() # Get the json-encoded content from response
        if response_json["response"] !=[]:
            i=0
            for item in response_json["response"]:
                i+=1
                ip_list.append([i,"host",item["hostIp"]])
            idx=i # idx(sequential number) will be used to tag host and network device
    except:
        print ("Something wrong, cannot get host IP list")
    # So far "ip_list" contains all hosts

    # Now get network device and append it to the list
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device")
        print ("Status: of GET /network-device ",resp.status_code)  # This is the http request status
        response_json = resp.json() # Get the json-encoded content from response
        if response_json["response"] !=[]:
            for item in response_json["response"]:
                idx+=1
                ip_list.append([idx,"network device",item["managementIpAddress"]])
    except:
        print ("Something wrong ! Cannot get network-device IP list !")
    # Now "ip_list" should have hosts and network-devices

    if ip_list !=[]:
        return ip_list
    else:
        print ("There is no any host or network device !")
        sys.exit()
        
print (tabulate(get_host_and_device(),headers=['number','type','ip'],tablefmt="rst"))


Executing GET 'https://sandboxapicem.cisco.com/api/v1/host'

GET 'host' Status:  200 

Status of GET /host:  200

Executing GET 'https://sandboxapicem.cisco.com/api/v1/network-device'

GET 'network-device' Status:  200 

Status: of GET /network-device  200
========  ==============  =============
  number  type            ip
========  ==============  =============
       1  host            10.1.15.117
       2  host            10.2.1.22
       3  host            10.1.12.20
       4  network device  165.10.1.39
       5  network device  10.1.14.3
       6  network device  10.2.1.17
       7  network device  10.2.2.1
       8  network device  10.2.2.2
       9  network device  218.1.100.100
      10  network device  10.1.12.1
      11  network device  10.1.7.1
      12  network device  10.1.10.1
      13  network device  10.255.1.5
      14  network device  10.1.11.1
      15  network device  10.1.2.1
      16  network device  10.1.4.2
      17  network device  10.1.14.2
========  ======

### Get path trace with flowAnalysisId

In [29]:
def check_status(arg,arg1):
    """
    Non-blocking wait function to check POST /flow-analysis status:
    INPROGRESS, COMPLETED, FAILED

    Parameters
    ----------
    arg (str) : status of POST /flow-analysis
    arg1 (str): flowAnalysisId from POST /flow-analysis
    Return:
    -------
    None
    """
    status = arg
    flowAnalysisId = arg1
    count = 0
    while status != "COMPLETED":
        if status == "FAILED":
            print("Unable to find full path. No traceroute or netflow information found. Failing path calculation.")
            print("\n------ End of path trace ! ------")
            sys.exit()
        print ("\nTask is not finished yet, sleep 1 second then try again")
        time.sleep(1)
        count += 1
        if count > 20: # timeout after ~ 20 seconds
            print ("\nScript time out, no routing path was found. Please try using different source and destination !")
            print("\n------ End of path trace ! ------")
            sys.exit()
        try:
            r = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="flow-analysis/"+flowAnalysisId)
            response_json = r.json()
            # print ("Response from GET /flow-analysis/"+flowAnalysisId,json.dumps(response_json,indent=4))
            status = response_json["response"]["request"]["status"]
            print ("\n**** Check status here: ",status," ****\n")
        except:
            # Something is wrong
            print ("\nSomething is wrong when executing get /flow-analysis/{flowAnalysisId}")
            sys.exit()
    print ("Response from GET /flow-analysis/"+flowAnalysisId,json.dumps(response_json,indent=4))
    print("\n------ End of path trace ! ------")

def get_host_and_device():
    """
    This function returns a list of all hosts and network devices with a number tag.

    Return:
    ------
    list: a list of all hosts and network devices with a number tag
    """
    ip_list=[]
    idx=0
    # Create a list of host and network device
    # Get host
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="host")
        response_json = resp.json() # Get the json-encoded content from response
        i=0
        if response_json["response"] !=[]:
            for item in response_json["response"]:
                i+=1
                ip_list.append([i,"host",item["hostIp"]])
            idx=i # This idx(sequential number) will be used to tag host and network device
                  # So far this number = the number of hosts
    except:
        print ("Something wrong, cannot get host IP list")

    # Now get network device and append it to the list
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device")
        print ("Status: of GET /network-device ",resp.status_code)  # This is the http request status
        response_json = resp.json() # Get the json-encoded content from response
        if response_json["response"] !=[]:
            for item in response_json["response"]:
                idx+=1
                ip_list.append([idx,"network device",item["managementIpAddress"]])
    except:
        print ("Something wrong ! Cannot get network-device IP list !")
    # Now "ip_list" should have hosts and network-devices
    return ip_list

def select_ip(prompt,ip_list,idx):
    """
    This function return a element that user selected from a tabular list

    Parameters
    ----------
    prompt: str
        message to prompt user
    ip_list: list
        a list with idx that user can make a selection
    idx: int
        position of element to retrieve from list

    Return:
    -------
    str: user selected IP address
    """

    ip =""
    while True:
        user_input = input(prompt)
        user_input= user_input.lstrip() # Ignore leading space
        if user_input.lower() == 'exit':
            sys.exit()
        if user_input.isdigit():
            if int(user_input) in range(1,len(ip_list)+1):
                ip = ip_list[int(user_input)-1][idx] # The idx is the position of IP
                return ip
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop


ip_idx = 2
nd_list = get_host_and_device()
if len(nd_list) < 2:
    print ("We need at least 2 host or network-device to perform path trace!")
    sys.exit()

print (tabulate(nd_list,headers=['number','type','ip'],tablefmt="rst"))
print ("*** Please note that not all source/destination ip pair will return a path - no route. ! *** \n")
s_ip = select_ip('=> Select a number for the source IP from above list: ',nd_list,ip_idx) # ip_idx (=2) is the position of IP in the list
d_ip = select_ip('=> Select a number for the destination IP from above list: ',nd_list,ip_idx) # ip_idx (=2) is the position of IP in the list
# Now we have the souce and destination IPs we can use them to POST /flow-analysis
path_data = {"sourceIP": s_ip, "destIP": d_ip} # JSON input for POST /flow-analysis - simplify one
# path_data= {"sourceIP":s_ip,"destIP":d_ip,"periodicRefresh":False,"inclusions":["QOS-STATS","INTERFACE-STATS","DEVICE-STATS","PERFORMANCE-STATS","ACL-TRACE"]}
# above JSON will trigger the respone to include stats of QoS, interface, device and ACL trace
r = apicem.post(APICEM_IP, VERSION, USERNAME, PASSWORD, api="flow-analysis",data=path_data) # Execute POST /flow-analysis
response_json = r.json()
print ("Response from POST /flow-analysis:\n",json.dumps(response_json,indent=4))
try:
   flowAnalysisId = response_json["response"]["flowAnalysisId"]
except:
    print ("\n For some reason cannot get flowAnalysisId")
    sys.exit()

###########################################################
# Check status of POST /flow-analysis - non-blocking wait #
###########################################################
thread = threading.Thread(target=check_status, args=('',flowAnalysisId,)) # Passing <status = ''>
thread.start()


Executing GET 'https://sandboxapicem.cisco.com/api/v1/host'

GET 'host' Status:  200 


Executing GET 'https://sandboxapicem.cisco.com/api/v1/network-device'

GET 'network-device' Status:  200 

Status: of GET /network-device  200
========  ==============  =============
  number  type            ip
========  ==============  =============
       1  host            10.1.15.117
       2  host            10.2.1.22
       3  host            10.1.12.20
       4  network device  165.10.1.39
       5  network device  10.1.14.3
       6  network device  10.2.1.17
       7  network device  10.2.2.1
       8  network device  10.2.2.2
       9  network device  218.1.100.100
      10  network device  10.1.12.1
      11  network device  10.1.7.1
      12  network device  10.1.10.1
      13  network device  10.255.1.5
      14  network device  10.1.11.1
      15  network device  10.1.2.1
      16  network device  10.1.4.2
      17  network device  10.1.14.2
========  ==============  =============
**

## LAB 3. Policy Labs 🥑

In [32]:
def prettyPrint(text="",json_object=None):
        resp = json_object.json() # Get the json-encoded content from response
        print(text,json.dumps(resp,indent=4))    # This is the entire response from the query

### 3.1 Post-policy-tag

In [33]:
def create_policy_tag(tag_json):
    try:
        resp = apicem.post(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag", data=tag_json)
        prettyPrint("Response:\n", resp)
    except:
        print ("Something wrong with POST /policy/tag !")

# Ask user's input
# In the loop until input is not null or is 'exit'
print ("** Tag must only include letters, numbers, underscore and hyphen, no space between two words **")
while True:
    pTag = input('=> Enter policy tag name that you like to create: ')
    pTag = pTag.lstrip() # Ignore leading space
    if pTag.lower() == 'exit':
        sys.exit()
    if pTag == "":
        print ("Oops! Policy tag name cannot be NULL please try again or enter 'exit'")
    else:
        break

# JSON for "POST policy/tag" request, taking user's input as tag name
tag_json = {
    "policyTag": pTag
}

create_policy_tag(tag_json) # Create tag function

** Tag must only include letters, numbers, underscore and hyphen, no space between two words **
=> Enter policy tag name that you like to create: goodcisco

Executing POST 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

POST 'policy/tag' Status:  202 

Response:
 {
    "response": {
        "taskId": "ce432eb9-0a23-4304-a9c6-921780dbabfc",
        "url": "/api/v1/task/ce432eb9-0a23-4304-a9c6-921780dbabfc"
    },
    "version": "1.0"
}


### 3.2 Get policy tag

In [35]:
def get_policy_tag():
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag")
        prettyPrint("Response:\n", resp)
    except:
        print ("Something wrong with GET /policy/tag !")


get_policy_tag()


Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

GET 'policy/tag' Status:  200 

Response:
 {
    "response": [
        {
            "policyTag": "YESLAB"
        },
        {
            "policyTag": "VK_POLICY"
        },
        {
            "policyTag": "luisga"
        },
        {
            "policyTag": "Vijoy_Added_this"
        },
        {
            "policyTag": "yeslab_hello"
        },
        {
            "policyTag": "Student_Added_This"
        },
        {
            "policyTag": "lala"
        },
        {
            "policyTag": "UTCN"
        },
        {
            "policyTag": "Bryan_gonz"
        },
        {
            "policyTag": "vanakkad"
        },
        {
            "policyTag": "goodcisco"
        },
        {
            "policyTag": "mahovich"
        },
        {
            "policyTag": "greatcisco"
        },
        {
            "policyTag": "RemoteSite"
        },
        {
            "policyTag": "lpj"
        },
 

### 3.3 Post policy tag association

In [36]:
def select_device_id():
    device=[]
    # Create a list of network devices
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="network-device")
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        device = response_json["response"] # The network-device
    except:
        print ("Something wrong, cannot get network device information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        print (resp.text)
        sys.exit()

    if device == []:
        print ("Oops! No device was found ! Discover network device first.")
        sys.exit()

    device_list = []
    # Extracting attributes and add a counter to an iterable
    idx=0
    for item in device:
        idx+=1
        device_list.append([idx,item["hostname"],item["managementIpAddress"],item["type"],item["instanceUuid"]])
    if device_list == []:
        print ("There is no network-device can be used to associate with policy tag !")
        sys.exit()
    # Pretty print tabular data, needs 'tabulate' module
    print (tabulate(device_list, headers=['number','hostname','ip','type'],tablefmt="rst"),'\n')

    # Ask user's selection
    # Find out network device with selected ip or hostname, index 4 is the network device id
    # In the loop until 'id' is assigned or user enter 'exit'
    net_id = ""
    device_id_idx = 4 # Network device ip index in the list
    while True:
        user_input = input('Select a number for the device from the list to add policy tag: ')
        user_input= user_input.lstrip() # Ignore leading space
        if user_input.lower() == 'exit':
            sys.exit()
        if user_input.isdigit(): # Make sure user's input in in range
            if int(user_input) in range(1,len(device_list)+1):
                net_id = device_list[int(user_input)-1][device_id_idx] # The device_id_idx is the position of id
                return net_id
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

######## select a policy tag to associate with device ##########

def select_policy_tag():
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag") # get policy tag
        response_json = resp.json()
        tag = response_json["response"] # policy tags
    except:
        print ("Something wrong, cannot get host policy tag")
    if tag ==[] :
        print ("No policy tag was found, create policy tag first !")
        sys.exit()
    i=0
    tag_list = []
    for item in tag:
        i+=1
        tag_list.append([i,item["policyTag"]])
    print (tabulate(tag_list, headers=['#','Policy Tag'],tablefmt="rst"),'\n')
    pTag=""
    # Ask user's input
    # In the loop until tag is selected or user select 'exit'
    while True:
        tag_num = input('=> Select a number for the tag from the list: ')
        tag_num = tag_num.lstrip() # ignore leading space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit(): # make sure digit is entered
            if int(tag_num) in range(1,len(tag)+1): # make sure digit entered is in range
                pTag=tag[int(tag_num)-1]["policyTag"]
                return pTag
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

def post_association(tag,n_id):
    #JSON for POST /policy/tag/association
    r_json = {
        "policyTag":tag,
        "networkDevices":[{"deviceId":n_id}]
    }
    # POST "/policy/tag/association" API
    try:
        resp = apicem.post(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association",data=r_json)
        prettyPrint("Response:\n", resp)
    except:
        print ("\nSomething is wrong when executing POST /policy/tag/association")


net_id = select_device_id() # getting network device id
tag = select_policy_tag() # getting policy tag
post_association(tag,net_id) # create association


Executing GET 'https://sandboxapicem.cisco.com/api/v1/network-device'

GET 'network-device' Status:  200 

========  =============================  =============  ==============================================
  number  hostname                       ip             type
========  =============================  =============  ==============================================
       1  AHEC-2960C1                    165.10.1.39    Cisco Catalyst 2960C-8PC-L Switch
       2  AP7081.059f.19ca               10.1.14.3      Cisco 3500I Unified Access Point
       3  Branch-Access1                 10.2.1.17      Cisco Catalyst 29xx Stack-able Ethernet Switch
       4  Branch-Router1                 10.2.2.1       Cisco 2911 Integrated Services Router G2
       5  Branch-Router2                 10.2.2.2       Cisco 2911 Integrated Services Router G2
       6  Branch2-Router.yourdomain.com  218.1.100.100  Cisco 2911 Integrated Services Router G2
       7  CAMPUS-Access1                 10.1.12.1  

### 3.4 Get policy tag association

In [37]:
def get_tag_association():
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association")
        response_json = resp.json()
        tag = response_json["response"] # Policy tags
    except:
        print ("Something wrong with getting policy tag !")
        sys.exit()

    # If there is any policy tag, the response will show what network device is tagged
    if tag ==[]:
        print ("No Policy tag is found")
        sys.exit()
    else:
        tag_list = []
        i=0
        for item in tag:
            if "policyTag" in item:
                i+=1
                if item["networkDevices"] != []: # If there is at least one network device associated
                    for item1 in item["networkDevices"]: # There could be more than one network devices associated with the same tag
                        # Adding number in the beginning of each row
                        tag_list.append([i,item["policyTag"],item1["deviceName"],item1["deviceIp"],item1["deviceId"]])
                else:
                    tag_list.append([i,item["policyTag"],"","",""])
        if tag_list == []:
            print ("No policy tag association is found, nothing to show")
            sys.exit()

    print (tabulate(tag_list, headers=['#','Policy Tag associated with','Device Name','Device IP','Deice ID'],tablefmt="rst"),'\n')


get_tag_association()


Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag/association'

GET 'policy/tag/association' Status:  200 

===  ============================  =============================  =============  ====================================
  #  Policy Tag associated with    Device Name                    Device IP      Deice ID
===  ============================  =============================  =============  ====================================
  1  YESLAB
  2  VK_POLICY                     Branch-Router2                 10.2.2.2       6ce631db-9212-4587-867f-b8f3aed1702d
  2  VK_POLICY                     CAMPUS-Core1                   10.1.7.1       30d39b18-9ada-4148-ad6c-2ee20975b845
  2  VK_POLICY                     Campus-WLC-5508                10.1.14.2      ae19cd21-1b26-4f58-8ccd-d265deabb6c3
  3  luisga
  4  Vijoy_Added_this
  5  yeslab_hello
  6  Student_Added_This
  7  lala
  8  UTCN                          Branch-Router1                 10.2.2.1       0dd240fd-5cca-477

### 3.5 Post application

In [39]:
def post_app(app_json):
    ########## Get category id ##########
    # We need to know category id in the JSON of "POST /application" API
    # If the DB initialize, these ids will change. So get id dynamically.

    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="category") # The response (result) from "GET 'category" request
        response_json = resp.json() # Get the json-encoded content from response
        categories = response_json["response"] # category
    except:
        print ("Something wrong, cannot get category information")
        sys.exit()
    # Find the category id for the category name is used in JSON
    for item in categories:
        if item["name"] == app_json["category"]:
            app_json["categoryId"] = item["id"]

    # Populate user input to JSON object
    app_json["helpString"] = pApp
    app_json["name"] = pApp
    app_json["ignoreConflict"] = True

    # Important: Convert to list -- this API requires that
    app_json = [app_json]

    # POST application url
    try:
        resp = apicem.post(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application", data=app_json)
        prettyPrint("Response:\n",resp)
    except:
        print ("Something wrong with POST /application !")

##########################################################################

# Creating JSON object for the  POST request

app_json = {
    "trafficClass":"BULK_DATA",
    "helpString":"",
    "name":"",
    "appProtocol": "tcp/udp",
    "udpPorts": "8888",
    "tcpPorts": "8888",
    "pfrThresholdJitter":1,
    "pfrThresholdLossRate":50,
    "pfrThresholdOneWayDelay":500,
    "pfrThresholdJitterPriority":1,
    "pfrThresholdLossRatePriority":2,
    "pfrThresholdOneWayDelayPriority":3,
    "category":"other",
    "subCategory":"other",
    "categoryId":"",
    "longDescription": "custom application",
    "ignoreConflict":True
    }

########## Ask user to enter application name ##########
# In the loop until input is not null or is 'exit'

while True:
    print ("** The name only include letters, numbers, underscore and hyphen, no space between two words **")
    pApp = input('=> Enter application name that you like to create: ')
    pApp = pApp.lstrip() # Ignore leading space
    if pApp.lower() == 'exit':
        sys.exit()
    if pApp == "":
        print ("Oops! Application name cannot be NULL please try again or enter 'exit'")
    else:
        break
# Everything is OK so far, initialize apicem instance and create application
post_app(app_json)

** The name only include letters, numbers, underscore and hyphen, no space between two words **
=> Enter application name that you like to create: cisco_yeslab

Executing GET 'https://sandboxapicem.cisco.com/api/v1/category'

GET 'category' Status:  200 


Executing POST 'https://sandboxapicem.cisco.com/api/v1/application'

POST 'application' Status:  202 

Response:
 {
    "response": {
        "taskId": "02c06734-3059-4dcf-974c-0941b4aeb0a9",
        "url": "/api/v1/task/02c06734-3059-4dcf-974c-0941b4aeb0a9"
    },
    "version": "1.0"
}


### 3.6 Get custom application

In [40]:
def get_custom_app():
    app = []
    # Set "isCustom = True" so the API will return only custom application(s)
    params={"isCustom":True}
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application",params=params) # The response from "GET /application" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        app = response_json["response"]
    except:
        print ("Something wrong, cannot get application information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        sys.exit()

    # Make sure there is at least one application
    if app == []:
        print ("There is no custom application !")
        sys.exit()

    app_list = []
    # Extracting attributes
    for item in app:
        app_list.append([item["name"],item["instanceUuid"]])
    # Return a list of custom application with name and id
    return app_list


a_list = get_custom_app()
if a_list !=[]:
    print ("*************  All custom applications *************\n")
    print (tabulate(a_list, headers=['custom application','application id'],tablefmt="rst"),'\n')
else:
    print ("*************  There is no custom applications ! *************\n")


Executing GET 'https://sandboxapicem.cisco.com/api/v1/application'

GET 'application' Status:  200 

*************  All custom applications *************

====================  ====================================
custom application    application id
====================  ====================================
devent_app            11c0a656-3bba-442b-99c7-f9a045980077
cisco_yeslab          9648d3d1-75af-4d3d-aa1b-0fcfe70a6954
====================  ==================================== 



### 3.7 Policy preview

In [ ]:
def enter_policy_name():
    check_name = True
    p_name=""
    while check_name:
        p_name = input('=> Enter the policy name that you like to create for preview: ')
        p_name = p_name.lstrip() # Ignore leading space
        if p_name.lower() == 'exit':
            sys.exit()
        if p_name == "":
            print ("Oops! Policy name cannot be NULL please try again or enter 'exit'")
        else: # Check if name is used
            check_name = False
            try:
                resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy") # The response (result) from "GET /policy/" request
                response_json = resp.json() # Get the json-encoded content from response
                policy = response_json["response"]
            except:
                print ("Something wrong, cannot get policy information")
                sys.exit()
            for item in policy:
                if p_name == item["policyName"]:
                    print ("This policy name exists, please type in different name !")
                    check_name = True
                    break
    return p_name

########### Ask user to select a policy Business-Relevance ##############

def select_relevance():
    relevancy_select = [[1,"Business-Relevant"],[2,"Business-Irrelevant"],[3,"Default"]]
    relevancy_tag = ['-BR','-IR','-D']
    print (tabulate(relevancy_select, headers=['#','Policy Business Relevancy'],tablefmt="rst"),'\n')

    relevanceLevel = 'Default'
    # In the loop until tag is selected or user select 'exit'
    while True:
        tag_num = input('=> Enter a number above for policy Business Relevancy: ')
        tag_num = tag_num.lstrip() # Ignore leading space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit():
            if int(tag_num) in range(1,len(relevancy_select)+1):
                relevanceTag = relevancy_tag[int(tag_num)-1]
                relevanceLevel = relevancy_select[int(tag_num)-1][1]
                break
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

    return [relevanceLevel,relevanceTag]


def get_tag_association():
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association")
        response_json = resp.json()
        tag = response_json["response"] # Policy tags
    except:
        print ("Something wrong with getting policy tag !")
        sys.exit()

    # If there is any policy tag, the response will show what network device is tagged
    if tag ==[]:
        print ("No Policy tag is found")
        sys.exit()
    else:
        tag_list = []
        i=0
        for item in tag:
            if "policyTag" in item:
                if item["networkDevices"] != []: # If there is at least one network device associated
                    for item1 in item["networkDevices"]: # There could be more than one network device associated with the same tag
                        # i - Adding number in the beginning of each row
                        i+=1
                        tag_list.append([i,item["policyTag"],item1["deviceName"],item1["deviceIp"],item1["deviceId"]])
                else:
                    i+=1
                    tag_list.append([i,item["policyTag"],"","",""])
        if tag_list == []:
            print ("No policy tag association is found, nothing to show")
            sys.exit()
    return (tag_list)

########### Ask user to select a policy tag name ##############

def select_tag():
    tag_list= get_tag_association()
    print (tabulate(tag_list, headers=['#','Policy Tag associated with','Device Name','Device IP','Deice ID'],tablefmt="rst"),'\n')

    # In the loop until tag is selected or user select 'exit'
    tag_name_idx=1
    nd_id_idx = 4
    while True:
        tag_num = input('=> Select a policy tag that is associated with network device : ')
        tag_num = tag_num.lstrip() # Ignore leading space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit():
            if int(tag_num) in range(1,len(tag_list)+1):
                nd_id = tag_list[int(tag_num)-1][nd_id_idx]
                if nd_id == "":
                    print ("Oops! This policy tag is not associated with any network device, please try again or enter 'exit'")
                else:
                    tag_name = tag_list[int(tag_num)-1][tag_name_idx]
                    break
            else:
                print ("Oops! Number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! Input is not a digit, please try again or enter 'exit'")
    # End of while loop
    return [tag_name,nd_id]

########## Select an application and retrieve its id #################
def select_app():
    print ("** Retrieving applications may take a while, please wait......... **\n")
    app = []
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application") # The response (result) from "GET /application" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        app = response_json["response"]
    except:
        print ("Something wrong, cannot get application information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        sys.exit()

    # Make sure there is at least one application
    if app == []:
        print ("Something wrong for retrieving applications!")
        sys.exit()

    app_list = []
    # Extracting attributes
    for item in app:
         app_list.append([item["name"],item["instanceUuid"]])
    # Show all NBAR2 applications
    # Pretty print tabular data, needs 'tabulate' module

    print ("-------------  All default applications -------------")
    print (tabulate(app_list, headers=['application','id'],tablefmt="rst"),'\n')

    app = []
    params={"isCustom":True}
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application",params=params) # The response (result) from "GET /application" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        app = response_json["response"]
    except:
        print ("Something wrong, cannot get application information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        sys.exit()

    # Make sure there is at least one custom application
    if app == []:
        print ("** There is no custom application, select one from default ! **\n")
    else:
        short_list = []
        # Extracting attributes
        for item in app:
            short_list.append([item["name"],item["instanceUuid"]])
        print ("*************  All custom applications *************")
        print (tabulate(short_list, headers=['custom application','id'],tablefmt="rst"),'\n')

    # Ask user's select application in order to retrieve its id
    # In the loop until 'id' is assigned or user select 'exit'

    app_id = ""
    select = True
    while select:
        app_name = input('\n=> Enter application name from above(default or custom,case-sensitive) to create policy: ')
        app_name = app_name.lstrip() # Ignore leading space
        if app_name.lower() == 'exit':
            sys.exit()
        for item in app_list:
            if app_name == item[0]: # If user_input(application name) is matched
                app_id = item[1]    # Index 1 is the application id
                select = False
                break
        if app_id == "":
            print ("Oops! application was not found, please try again or enter 'exit'")
    # End of while loop

    return [app_name,app_id]

def get_file_with_id(id):

    try:
        resp=apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="file/"+id) # The response (result) from "GET /file/{id}" request
        status = resp.status_code
        # print (resp.text)
    except:
        print ("Something wrong, cannot get file information")
        sys.exit()

    if status != 200:
        print ("Response status :",status)
        print (resp.text)
        sys.exit()

    return  resp.text


policy_name = enter_policy_name() # Policy name for preview from user's input
relevance = select_relevance()
relevanceLevel = relevance[0]
tag = select_tag() # Select a policy tag scope
tag_name = tag[0]
net_id = tag[1]
app = select_app() # Select an application
app_name = app[0]
app_id = app[1]

# JSON object for POST /policy/preview

preview_json = {
    "policies":[
         {
         "policyName": policy_name,
         "policyOwner": "admin",
         "policyPriority": 4095,
         "resource": {
             "applications": [{"appName": app_name,"id": app_id}]
          },
          "actions":["SET_PROPERTY"],
          "policyScope": tag_name,
          "actionProperty": {"relevanceLevel": relevanceLevel}
         }
     ],
     "networkDeviceIds": [
          net_id
     ],
    "state": "ENABLE_DEVICE"
}

########## Creating policy preview #############
params={"policyScope" : tag_name}
try:
    myapicem.post(api="policy/preview", params=params,data=preview_json,printOut=True)
except:
    print ("Something wrong with POST policy/preview")
    sys.exit()
print ("Generating policy preview, please wait.....")
time.sleep(2) # It take a little time to generate preview so wait couple of seconds here.
count = 0
preview_created = False
loop = True
while loop:
    time.sleep(1)
    count += 1
    print ("Generating policy preview, please wait.....")
    try:
        # Preview for this policyScope
        resp=apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview",params=params)
        prettyPrint("Response:\n", resp)
        response_json = resp.json()
        preview = response_json["response"] # Policy tags
    except:
        print ("Something wrong with GET policy preview")
        sys.exit()
    if preview == []:
        print ("Something wrong with POST policy/preview, preview is not created")
        sys.exit()
    for item in preview: # Entire response from GET policy/preview
        for item1 in item["policies"]:
            if item1["policyName"] == policy_name: # Make sure preview is created, won't see the policy name if it's not created
                preview_created = True
                if item["deviceConfigs"] != []:
                    for item2 in item["deviceConfigs"]:
                        if item2["status"] == "FAILURE":
                            print ("*** Dry-Run Failed to Generate CLIs ***")
                            print (item2["failureReason"])
                            print ("Not thing to preview so deleting this request .....")
                            # Since nothing to view so delete this preview
                            apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview/"+item["id"])
                            sys.exit()
                        if item2["status"] == "SUCCESS":
                            file_id = (item2["fileId"])[13:] # Skip "/api/v1/file/"
                            loop = False
                            break
                        if count > 30: # Timeout after ~ 30 seconds
                            loop = False
                            if "fileId" in item2 : # The fileId may not show immediately
                                print ("Warning: May only have partial preview content !")
                                file_id = (item2["fileId"])[13:] # Skip "/api/v1/file/"
                                break
                            else:
                            # Since nothing to view so delete this preview
                                print ("\nScript time out, it takes to long to get the file id !")
                                apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview/"+item["id"])
                                sys.exit()
                else:
                    if count > 30: # Timeout after ~ 30 seconds
                        loop = False
                        print ("\nScript time out, it takes to long to get the file id !")
                        apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview/"+item["id"])
                        sys.exit()

    if not preview_created: # Policy name is not found in all existing policy preview, preview is not created
        print ( "Policy preview is not created, a preview may already exist for this policy scope - ",tag_name)
        sys.exit()
content = get_file_with_id(file_id)
print ("---------------- Policy Preview -----------------")
print (content)
print ("\nDeleting policy preview after presenting content.........\n")
apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview/"+item["id"])

=> Enter the policy name that you like to create for preview: cisco_yeslab

Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy'

GET 'policy' Status:  200 

===  ===========================
  #  Policy Business Relevancy
===  ===========================
  1  Business-Relevant
  2  Business-Irrelevant
  3  Default
===  =========================== 

=> Enter a number above for policy Business Relevancy: 3

Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag/association'

GET 'policy/tag/association' Status:  200 

===  ============================  =============================  =============  ====================================
  #  Policy Tag associated with    Device Name                    Device IP      Deice ID
===  ============================  =============================  =============  ====================================
  1  YESLAB
  2  VK_POLICY                     Branch-Router2                 10.2.2.2       6ce631db-9212-4587-867f-b8f3aed1702d
  3

### 3.8 Delete policy preview

In [ ]:
def select_policy_preview(ap):
    preview = [] # policy preview list
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview") # "GET /policy/preview" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        preview = response_json["response"]
    except:
        print ("Something wrong, cannot get policy information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        print (resp.text)
        sys.exit()

    # Make sure there is at least one policy preview
    if preview != [] :   # if response is not empty
        preview_list = []
        # Extracting attributes
        i=0
        for item in preview:
            for item1 in item["policies"]:
                i+=1
                preview_list.append([i,item1["policyName"],item1["policyScope"],item["id"]])
        # Show all policy preview
        # Pretty print tabular data, needs 'tabulate' module
        print (tabulate(preview_list, headers=["#",'policy name of preview','policy scope','id'],tablefmt="rst"),'\n')
    else:
        print ("There is no policy preview, nothing to delete !")
        sys.exit()

    ######## select a policy preview and return policy preview id #######
    # Ask user's input
    # In the loop until 'policy preview id' is assigned or user entered 'exit'

    id_idx = 3   # policy preview id index in the list
    name_idx = 1
    while True:
        user_input = input('=> Select a number for the policy to delete: ' )
        user_input= user_input.lstrip() # Ignore leading space
        if user_input.lower() == 'exit':
            sys.exit()
        if user_input.isdigit():
            if int(user_input) in range(1,len(preview_list)+1):
                preview_id = preview_list[int(user_input)-1][id_idx]
                policy_name = preview_list[int(user_input)-1][name_idx]
                return [policy_name,preview_id] # return value of this function
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

#### Delete Policy ####


preview_info=select_policy_preview()
print ("Deleting",preview_info[0],"....") # preview_info[0] = policy_name
try:
    # delete policy by policy preview id
    resp = apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/preview/"+preview_info[1]) # preview_info[1] = policy preview id
    prettyPrint("Response:\n",resp)
except:
    print ("Something wrong with deleting policy")
    sys.exit()

### 3.9 Post policy

In [ ]:
def enter_policy_name():
    check_name = True
    while check_name:
        policy_name = input('=> Enter policy name that you like to create: ')
        policy_name = policy_name.lstrip() # Ignore leading space
        if policy_name.lower() == 'exit':
            sys.exit()
        if policy_name == "":
            print ("Oops! Policy name cannot be NULL please try again or enter 'exit'")
        else: # Check if name is used
            check_name = False
            try:
                resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy") # The response (result) from "GET /policy" request
                response_json = resp.json() # Get the json-encoded content from response
                policy = response_json["response"]
            except:
                print ("Something wrong, cannot get policy information")
                sys.exit()
            for item in policy:
                if policy_name == item["policyName"]:
                    print ("This policy name exists, please type in different name !")
                    check_name = True
                    break
    return policy_name


########### Ask user to select a policy Business-Relevance ##############

def select_relevance():
    relevancy_select = [[1,'Business-Relevant'],[2,'Business-Irrelevant'],[3,'Default']]
    relevancy_tag = ['-BR','-IR','-D']
    print (tabulate(relevancy_select, headers=['#','Policy Business Relevancy'],tablefmt="rst"),'\n')

    relevanceLevel = 'Default'
    # In the loop until tag is selected or user select 'exit'
    while True:
        tag_num = input('=> Enter a number above for policy Business Relevancy: ')
        tag_num = tag_num.lstrip() # Ignore leading space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit():
            if int(tag_num) in range(1,len(relevancy_select)+1):
                relevanceTag = relevancy_tag[int(tag_num)-1]
                relevanceLevel = relevancy_select[int(tag_num)-1][1]
                break
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

    return [relevanceLevel,relevanceTag]

def get_tag_association():
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association")
        response_json = resp.json()
        tag = response_json["response"] # Policy tags
    except:
        print ("Something wrong with getting policy tag !")
        sys.exit()

    # If there is a policy tag, the response will show what network device is tagged
    if tag ==[]:
        print ("No Policy tag is found")
        sys.exit()
    else:
        tag_list = []
        i=0
        for item in tag:
            if "policyTag" in item:
                if item["networkDevices"] != []: # If there is at least one network device associated
                    for item1 in item["networkDevices"]: # There could be more than one network device associated with the same tag
                        i+=1
                        # Adding number in the beginning of each row
                        tag_list.append([i,item["policyTag"],item1["deviceName"],item1["deviceIp"],item1["deviceId"]])
                else:
                    i+=1
                    tag_list.append([i,item["policyTag"],"","",""])
        if tag_list == []:
            print ("No policy tag association is found, nothing to show")
            sys.exit()
    return (tag_list)

########### Ask user to select a policy tag name ##############

def select_tag():
    tag_list= get_tag_association()
    print (tabulate(tag_list, headers=['#','Policy Tag associated with','Device Name','Device IP','Deice ID'],tablefmt="rst"),'\n')

    # In the loop until tag is selected or user select 'exit'
    tag_name_idx = 1
    nd_id_idx = 4
    while True:
        tag_num = input('=> Select a policy tag that is associated with network device : ')
        tag_num = tag_num.lstrip() # Ignore leading space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit():
            if int(tag_num) in range(1,len(tag_list)+1):
                nd_id = tag_list[int(tag_num)-1][nd_id_idx]
                if nd_id == "":
                    print ("Oops! This policy tag is not associated with any network device, please try again or enter 'exit'")
                else:
                    tag_name = tag_list[int(tag_num)-1][tag_name_idx]
                    break
            else:
                print ("Oops! Number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! Input is not a digit, please try again or enter 'exit'")
    # End of while loop
    return tag_name

########## Select an application and retrieve its id #################
def select_app():
    print ("** Retrieving applications may take a while, please wait......... **\n")
    app = []
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application") # The response (result) from "GET /application" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        app = response_json["response"]
    except:
        print ("Something wrong, cannot get application information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        sys.exit()

    # Make sure there is at least one application
    if app == []:
        print ("Something wrong for retrieving applications!")
        sys.exit()

    app_list = []
    # Extracting attributes
    for item in app:
         app_list.append([item["name"],item["instanceUuid"]])
    # Show all NBAR2 applications
    # Pretty print tabular data, needs 'tabulate' module

    print ("-------------  All default applications -------------")
    print (tabulate(app_list, headers=['application','id'],tablefmt="rst"),'\n')

    app = []
    params={"isCustom":True}
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application",params=params) # The response (result) from "GET /application" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        app = response_json["response"]
    except:
        print ("Something wrong, cannot get application information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        sys.exit()

    # Make sure there is at least one custom application
    if app == []:
        print ("** There is no custom application, select one from default ! **\n")
    else:
        short_list = []
        # Extracting attributes
        for item in app:
            short_list.append([item["name"],item["instanceUuid"]])
        print ("*************  All custom applications *************")
        print (tabulate(short_list, headers=['custom application','id'],tablefmt="rst"),'\n')

    # Ask user's select application in order to retrieve its id
    # In the loop until 'id' is assigned or user select 'exit'

    app_id = ""
    select = True
    while select:
        app_name = input('=> Enter application name from above(default or custom,case-sensitive) to create policy: ')
        app_name = app_name.lstrip() # Ignore leading space
        if app_name.lower() == 'exit':
            sys.exit()
        for item in app_list:
            if app_name == item[0]: # if user_input(application name) is matched
                app_id = item[1]    # index 1 is the application id
                select = False
                break
        if app_id == "":
            print ("Oops! application was not found, please try again or enter 'exit'")
    # End of while loop

    return [app_name,app_id]



policy_name = enter_policy_name() # First enter policy name
relevance = select_relevance()
relevanceLevel = relevance[0]
# policy_name = policy_name + relevance[1] # append relevance abbreviation to the policy name
tag_name = select_tag() # select a policy tag
app = select_app() # select an application
app_name = app[0]
app_id = app[1]

# JSON object for POST /policy
#  "SET_PROPERTY"
policy_json = [{
"policyName": policy_name,
"policyOwner": "devnetuser",
"policyPriority": 4095,
"resource": {
    "applications": [{
        "appName": app_name,
        "id": app_id
    }]
},
"actions":[
      "SET_PROPERTY"
],
"policyScope": tag_name,
"actionProperty": {
    "relevanceLevel": relevanceLevel
    }
}]

########## Creating policy #############
print ("\nCreating policy with a single application.........\n")
try:
    resp = apicem.post(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy", data=policy_json)
    prettyPrint("Response:\n",resp)
except:
    print ("Something wrong with POST policy")
    sys.exit()

### 3.10 Get policy

In [ ]:
def get_policy():
    # policy list
    policy = []
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy") # The response (result) from "GET /policy" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        policy = response_json["response"] # network-device
    except:
        print ("Something wrong, cannot get policy information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        print (resp.text)
        sys.exit()

    # Make sure there is at least one policy

    if policy == [] :
        print ("No policy was found !")
        sys.exit()
    # if response is not empty
    policy_list = []
    # Extracting attributes
    for item in policy:
        policy_list.append([item["policyName"],item["instanceUuid"]])
    # Show all policies
    # Pretty print tabular data, needs 'tabulate' module
    print (tabulate(policy_list, headers=['policy','id'],tablefmt="rst"),'\n')


get_policy()

### 3.11 Delete policy

In [ ]:
def select_policy():
    policy = [] # policy list
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy") # "GET /policy" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        policy = response_json["response"]
    except:
        print ("Something wrong, cannot get policy information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        print (resp.text)
        sys.exit()

    # Make sure there is at least one policy
    if policy != [] :   # if response is not empty
        policy_list = []
        # Extracting attributes
        i=0
        for item in policy:
            i+=1
            policy_list.append([i,item["policyName"],item["instanceUuid"]])
        # Show all policies
        # Pretty print tabular data, needs 'tabulate' module
        print (tabulate(policy_list, headers=["#",'policy','id'],tablefmt="rst"),'\n')
    else:
        print ("No policy was found !")
        sys.exit()

    print ("!!! BUSINESS_RELEVANT_CVD_Policy,DEFAULT_CVD_Policy,BUSINESS_IRRELEVANT_CVD_Policy !!!")
    print ("!!!                  These are default policies cannot be deleted                  !!!")
    print ("--------------------------------------------------------------------------------------")

    ######## select a policy and return policy name and policy id #######
    # Ask user's input
    # In the loop until 'policy id' is assigned or user entered 'exit'

    name_idx = 1 # policy name index in the list
    id_idx = 2   # policy id index in the list
    while True:
        user_input = input('=> Select a number for the policy to delete: ' )
        user_input= user_input.replace(" ","") # ignore space
        if user_input.lower() == 'exit':
            sys.exit()
        if user_input.isdigit():
            if int(user_input) in range(1,len(policy_list)+1):
                policy_name = policy_list[int(user_input)-1][name_idx]
                policy_id = policy_list[int(user_input)-1][id_idx]
                return [policy_name,policy_id] # return value of this function
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

#### Delete Policy ####


policy_info=select_policy()
print ("Deleting",policy_info[0],"....") # policy_info[0] = policy_name
try:
    # delete policy by policy id
    resp = apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/"+policy_info[1]) # policy_info[1] = policy_id
    prettyPrint("Response:\n",resp)
except:
    print ("Something wrong with deleting policy")
    sys.exit()

### 3.12 Delete custom application

In [ ]:
def select_application():
    app = []
    params={"isCustom":True} # filter, only retrieve custom application
    try:
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application",params=params) # The response (result) from "GET /application" request
        status = resp.status_code
        response_json = resp.json() # Get the json-encoded content from response
        app = response_json["response"]
    except:
        print ("Something wrong, cannot get application information")
        sys.exit()

    if status != 200:
        print ("Response status %s,Something wrong !"%status)
        sys.exit()

    custom_app = []
    if app != [] :   # if response is not empty
        # Extracting attributes
        idx=0
        for item in app:
            idx+=1 # adding numbers in the list
            custom_app.append([idx,item["name"],item["id"]])
        # Show all custom applications
        # Pretty print tabular data, needs 'tabulate' module

    if custom_app == []:
        print ("No custom NBAR2 application found, nothing to delete !")
        sys.exit()
    else:
        name_list=[]   # List of all custom application names
        app_in_policy=[] # list of all all custom applications which are used by policy
        for item in custom_app:
            name_list.append(item[1])
        # Iterate through all polices to find out if custom application is used
        resp= apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy")
        policy = resp.json()["response"]
        for item in policy:
            if "resource" in item:
                for item1 in item["resource"]["applications"]:
                    if item1["appName"] in name_list:
                        app_in_policy.append([item1["appName"],item["policyName"]])
        # Here, we check if there are applications used by policy
        # If there are applications used the we won't be able to delete
        i = 0
        policy_name_idx=2
        # Iterate custom application list
        for item in custom_app:
            match = False
            # Go Through 'in used' applications and insert policy name in the position 2 of the list
            for item1 in app_in_policy:
                print (item[1])
                if item[1] in item1:
                    match = True
                    policy_name=item1[1]
                    break
            if match:
                custom_app[i].insert(policy_name_idx,policy_name)
            else:
                custom_app[i].insert(policy_name_idx,"") # leave it blank if not used by any policy
                i=i+1
        print ("******** If application is used by policy it cannot be deleted ! *************")
        print (tabulate(custom_app, headers=['number','custom application','used by policy'],tablefmt="rst"),'\n')

    ######## Now let user to select an application and delete it #######
    # Ask user's input
    # In the loop until 'id' is assigned or user select 'exit'
    app_id = ""
    id_idx = 3 # #custom_app id is in position 3
    while True:
        user_input = input('=> Select a number for the application to delete:' )
        user_input= user_input.replace(" ","") # ignore space
        if user_input.lower() == 'exit':
            sys.exit()
        if user_input.isdigit():
            if int(user_input) in range(1,len(custom_app)+1):
                app_id = custom_app[int(user_input)-1][id_idx] #custom_app id is in position 3
                return app_id
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

#### Delete application ####

app_id=select_application(myapicem)  # get custom application id
try:
    resp = apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="application/"+app_id) # Delete application by application id
    prettyPrint("Response:\n",resp)
except:
    print ("Something wrong with deleting application")
    sys.exit()

### 3.13 Delete policy tag association

In [ ]:
def select_tag_association():
    # Get policy tag association

    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association")
        response_json = resp.json()
        tag = response_json["response"] # policy tag association
    except:
        print ("Something wrong with getting policy tag !")
        sys.exit()
    tag_list = []
    i=0
    for item in tag:
        if "policyTag" in item:
            if item["networkDevices"] != []:
                for item1 in item["networkDevices"]:
                    i+=1
                    tag_list.append([i,item["policyTag"],item1["deviceName"],item1["deviceIp"],item1["deviceId"]])
    if tag_list ==[]:
        print ("No policy tag association is found, nothing to delete")
        sys.exit()

    print ("The following are network devices that have policy tag")
    print (tabulate(tag_list, headers=['#','Policy Tag associated with','Device Name','Device IP'],tablefmt="rst"),'\n')


    # Ask user's input
    # In the loop until tag is selected or user select 'exit'
    tag_to_delete=""
    device_id_to_delete=""
    while True:
        tag_num = input('=> Enter a number from above to delete policy tag association: ')
        tag_num = tag_num.replace(" ","") # ignore space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit():
            if int(tag_num) in range(1,len(tag_list)+1):
                tag_to_delete=tag_list[int(tag_num)-1][1]
                device_id_to_delete=tag_list[int(tag_num)-1][4]
                break
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

    if tag_to_delete=="" or device_id_to_delete=="":
        print ("For some reason, tag name is NULL!")
        sys.exit()
    else:
        return  [tag_to_delete,device_id_to_delete]

########################## Delete policy tag association ########################


tag_id_list=select_tag_association()

params={"policyTag":tag_id_list[0],"networkDeviceId":tag_id_list[1]}
# To delete tag association needs to pass name of policy tag and network device id as parameters
try:
    resp = apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association/",params=params)
    prettyPrint("Response:\n",resp)
except:
    print ("Something wrong with deleting policy/tag/association")
    sys.exit()

### 3.14 Delete policy tag

In [ ]:
def select_tag():
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/count")
        response_json = resp.json()
        count = response_json["response"] # policy tags
    except:
        print ("Something wrong with getting policy tag count!")
        sys.exit()
    if count == 0 :
        print ("No policy tag is found, nothing to delete !")
        sys.exit()
    try:
        resp = apicem.get(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/association")
        response_json = resp.json()
        tag = response_json["response"] # policy tag association
    except:
        print ("Something wrong with GET policy/tag/association!")
        sys.exit()

    tag_list = []

    i=0
    for item in tag:
        if "policyTag" in item:
            if item["networkDevices"] == []:
                i+=1
                tag_list.append([i,item["policyTag"],"",""])
            else:
                for item1 in item["networkDevices"]:
                    i+=1
                    tag_list.append([i,item["policyTag"],item1["deviceName"],item1["deviceIp"]])


    print ("*** If policy tag is associated with network device, it cannot be deleted ***\n")
    print ("---------------- Select one with no network device attached -----------------\n")
    print (tabulate(tag_list, headers=['Number','Policy Tag associated with','Device Name','Device IP'],tablefmt="rst"),'\n')

    # Ask user's input
    # In the loop until tag is selected or user select 'exit'
    tag_to_delete=""
    tag_idx = 1 # 1 is the position of policy tag
    device_ip_idx = 3 #3 is the position of device IP
    while True:
        tag_num = input('=> Enter a number from above to delete policy tag: ')
        tag_num = tag_num.replace(" ","") # ignore space
        if tag_num.lower() == 'exit':
            sys.exit()
        if tag_num.isdigit():
            if int(tag_num) in range(1,len(tag_list)+1):
                tag_to_delete=tag_list[int(tag_num)-1][tag_idx] # 1 is the position of policy tag
                # to prevent user executing `DELETE /policy/tag` API not knowing actually fail to delete policy tag
                if tag_list[int(tag_num)-1][device_ip_idx] !="":
                    print("This tag is still associated with network device, select one with no network device attached !")
                else:
                    return tag_to_delete # OK to return policy tag name
            else:
                print ("Oops! number is out of range, please try again or enter 'exit'")
        else:
            print ("Oops! input is not a digit, please try again or enter 'exit'")
    # End of while loop

    if tag_to_delete=="":
        print ("For some reason, tag name is NULL!")
        sys.exit()

############################### Delete policy tag  ##############################

tag_to_delete = select_tag() # get the policy tag name
params={'policyTag':tag_to_delete} # to delete policy tag we need to pass tag name as parameter
try:
    resp = apicem.delete(APICEM_IP, VERSION, USERNAME, PASSWORD, api="policy/tag/",params=params)
    prettyPrint("Response:\n",resp)
except:
    print ("Something wrong with deleting policy/tag")
    sys.exit()